# Premier Reseau de Neurones avec PyTorch

> Ce notebook est un exemple pratique clef en main pour construire votre premier reseau de neurones avec PyTorch.
> Nous allons partir de zero et construire pas a pas un classificateur d'images capable de reconnaitre des chiffres manuscrits.

## Objectifs

- Comprendre les **tensors PyTorch** et leurs operations
- Charger et preparer un dataset d'images (**MNIST**)
- Construire un **Perceptron Multi-Couches** (MLP)
- Implementer une **boucle d'entrainement** complete
- **Evaluer** les performances du modele
- **Ameliorer** l'architecture avec Dropout et BatchNorm

### Pre-requis

```bash
# Installation des dependances (si necessaire)
uv add torch torchvision matplotlib numpy scikit-learn
```

In [ ]:
# === Imports ===
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Configuration de matplotlib pour de jolis graphiques
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Detection automatique du device (GPU si disponible, sinon CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device utilise : {device}")
print(f"Version de PyTorch : {torch.__version__}")

---
## Section 1 - Les Tensors PyTorch

Les **tensors** sont la structure de donnees fondamentale de PyTorch. Ce sont des tableaux multidimensionnels similaires aux arrays NumPy, mais avec deux avantages majeurs :
- Ils peuvent etre transferes sur **GPU** pour des calculs acceleres
- Ils supportent la **differentiation automatique** (autograd) pour la retropropagation

In [ ]:
# === Creation de tensors ===

# A partir d'une liste Python
tensor_liste = torch.tensor([1.0, 2.0, 3.0, 4.0])
print(f"Tensor depuis une liste : {tensor_liste}")
print(f"  Shape : {tensor_liste.shape}")
print(f"  Dtype : {tensor_liste.dtype}")
print()

# Tensors speciaux
zeros = torch.zeros(3, 4)        # Matrice 3x4 de zeros
ones = torch.ones(2, 3)          # Matrice 2x3 de uns
aleatoire = torch.randn(3, 3)    # Matrice 3x3 aleatoire (distribution normale)

print(f"Matrice de zeros (3x4) :\n{zeros}\n")
print(f"Matrice aleatoire (3x3) :\n{aleatoire}\n")

# === Operations sur les tensors ===
a = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
b = torch.tensor([[5.0, 6.0], [7.0, 8.0]])

# Operations element par element
print(f"Addition : \n{a + b}\n")
print(f"Multiplication element par element : \n{a * b}\n")

# Produit matriciel (tres utilise dans les reseaux de neurones)
produit = torch.matmul(a, b)  # ou a @ b
print(f"Produit matriciel (a @ b) : \n{produit}\n")

# === Reshape : redimensionner un tensor ===
# C'est crucial : une image 28x28 doit etre "aplatie" en vecteur de 784
image_simulee = torch.randn(28, 28)
vecteur = image_simulee.view(-1)  # -1 = calcul automatique de la taille
print(f"Image 28x28 -> Vecteur : {image_simulee.shape} -> {vecteur.shape}")

# === Device : transfert CPU <-> GPU ===
tensor_cpu = torch.randn(3, 3)
tensor_device = tensor_cpu.to(device)
print(f"\nTensor sur {tensor_device.device}")

---
## Section 2 - Le Dataset MNIST

**MNIST** est le "Hello World" du deep learning. Il contient **70 000 images** de chiffres manuscrits (0-9) en niveaux de gris, de taille 28x28 pixels.

- **60 000** images d'entrainement
- **10 000** images de test

Le dataset est telecharge automatiquement par `torchvision`.

In [ ]:
# === Chargement du dataset MNIST ===

# Transformation : convertir les images en tensors et normaliser les pixels [0,1] -> [-1,1]
transform = transforms.Compose([
    transforms.ToTensor(),                          # Convertit l'image PIL en tensor [0, 1]
    transforms.Normalize((0.1307,), (0.3081,))      # Normalise avec la moyenne et ecart-type de MNIST
])

# Telechargement et chargement des donnees
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# DataLoaders : chargent les donnees par lots (batches)
BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Nombre d'images d'entrainement : {len(train_dataset)}")
print(f"Nombre d'images de test : {len(test_dataset)}")
print(f"Taille d'un batch : {BATCH_SIZE}")
print(f"Nombre de batches d'entrainement : {len(train_loader)}")

# Inspectons un batch
images, labels = next(iter(train_loader))
print(f"\nShape d'un batch d'images : {images.shape}")
print(f"  -> {images.shape[0]} images, {images.shape[1]} canal, {images.shape[2]}x{images.shape[3]} pixels")
print(f"Shape des labels : {labels.shape}")

# === Visualisation d'echantillons ===
fig, axes = plt.subplots(2, 8, figsize=(16, 5))
fig.suptitle('Echantillons du dataset MNIST', fontsize=16, fontweight='bold')

for i, ax in enumerate(axes.flat):
    # On "denormalise" pour l'affichage
    img = images[i].squeeze()  # Retirer la dimension du canal (1, 28, 28) -> (28, 28)
    ax.imshow(img.numpy(), cmap='gray')
    ax.set_title(f'Label : {labels[i].item()}', fontsize=11)
    ax.axis('off')

plt.tight_layout()
plt.show()

# Distribution des classes
all_labels = [label for _, label in train_dataset]
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(all_labels, bins=range(11), edgecolor='black', align='left', color='steelblue')
ax.set_xlabel('Chiffre')
ax.set_ylabel('Nombre d\'images')
ax.set_title('Distribution des classes dans MNIST (entrainement)')
ax.set_xticks(range(10))
plt.tight_layout()
plt.show()

---
## Section 3 - Construire un Reseau de Neurones

Nous allons construire un **Perceptron Multi-Couches** (MLP) avec l'architecture suivante :

```
Image (28x28 = 784 pixels)
    |  Couche Lineaire (784 -> 128) + ReLU
    v
128 neurones
    |  Couche Lineaire (128 -> 64) + ReLU
    v
64 neurones
    |  Couche Lineaire (64 -> 10)
    v
10 sorties (une par chiffre 0-9)
```

**ReLU** (Rectified Linear Unit) est la fonction d'activation la plus courante : `f(x) = max(0, x)`

In [ ]:
# === Definition du modele MLP ===

class MLP(nn.Module):
    """Perceptron Multi-Couches pour la classification de chiffres MNIST."""

    def __init__(self):
        super(MLP, self).__init__()

        # Definir les couches du reseau
        self.couches = nn.Sequential(
            nn.Flatten(),           # (batch, 1, 28, 28) -> (batch, 784)
            nn.Linear(784, 128),    # Couche 1 : 784 entrees -> 128 neurones
            nn.ReLU(),              # Fonction d'activation
            nn.Linear(128, 64),     # Couche 2 : 128 -> 64 neurones
            nn.ReLU(),              # Fonction d'activation
            nn.Linear(64, 10),      # Couche de sortie : 64 -> 10 classes
        )

    def forward(self, x):
        """Propagation avant : passe les donnees a travers le reseau."""
        return self.couches(x)


# Instancier le modele et le deplacer sur le device
modele = MLP().to(device)
print(modele)

# Compter le nombre de parametres
nb_params = sum(p.numel() for p in modele.parameters())
nb_params_entrainables = sum(p.numel() for p in modele.parameters() if p.requires_grad)
print(f"\nNombre total de parametres : {nb_params:,}")
print(f"Nombre de parametres entrainables : {nb_params_entrainables:,}")

# Verification avec un batch factice
batch_test = torch.randn(4, 1, 28, 28).to(device)  # 4 images factices
sortie = modele(batch_test)
print(f"\nShape de la sortie : {sortie.shape}")
print(f"Sortie brute (logits) pour la 1ere image : {sortie[0].detach().cpu().numpy().round(3)}")
print(f"Prediction : chiffre {sortie[0].argmax().item()}")

---
## Section 4 - La Boucle d'Entrainement

L'entrainement d'un reseau de neurones suit toujours le meme schema :

1. **Propagation avant** (forward pass) : passer les donnees dans le reseau
2. **Calcul de la perte** (loss) : mesurer l'erreur entre la prediction et la verite
3. **Retropropagation** (backward pass) : calculer les gradients
4. **Mise a jour** des poids : l'optimiseur ajuste les parametres

Nous utilisons :
- **CrossEntropyLoss** : fonction de perte standard pour la classification multi-classes
- **Adam** : optimiseur adaptatif, tres utilise en pratique

> L'entrainement sur 5 epochs prend environ 1-2 minutes sur CPU.

In [ ]:
# === Configuration de l'entrainement ===

# Reinitialiser le modele pour un entrainement propre
modele = MLP().to(device)

# Fonction de perte : CrossEntropyLoss combine LogSoftmax + NLLLoss
critere = nn.CrossEntropyLoss()

# Optimiseur : Adam avec un taux d'apprentissage de 0.001
optimiseur = optim.Adam(modele.parameters(), lr=0.001)

# Nombre d'epochs (passages complets sur le dataset)
NB_EPOCHS = 5

# Historique pour les graphiques
historique = {
    'train_loss': [],
    'train_acc': [],
    'test_loss': [],
    'test_acc': []
}

# === Boucle d'entrainement ===
print("=" * 70)
print(f"{'Epoch':>6} | {'Train Loss':>12} | {'Train Acc':>10} | {'Test Loss':>12} | {'Test Acc':>10}")
print("=" * 70)

for epoch in range(NB_EPOCHS):
    # --- Phase d'entrainement ---
    modele.train()  # Mode entrainement
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        # Envoyer les donnees sur le device
        images, labels = images.to(device), labels.to(device)

        # 1. Propagation avant
        predictions = modele(images)

        # 2. Calcul de la perte
        loss = critere(predictions, labels)

        # 3. Retropropagation
        optimiseur.zero_grad()  # Remettre les gradients a zero
        loss.backward()         # Calculer les gradients

        # 4. Mise a jour des poids
        optimiseur.step()

        # Statistiques
        total_loss += loss.item() * images.size(0)
        _, predicted = torch.max(predictions, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = total_loss / total
    train_acc = 100 * correct / total

    # --- Phase d'evaluation ---
    modele.eval()  # Mode evaluation (desactive Dropout, etc.)
    total_loss_test = 0
    correct_test = 0
    total_test = 0

    with torch.no_grad():  # Pas besoin de calculer les gradients
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            predictions = modele(images)
            loss = critere(predictions, labels)

            total_loss_test += loss.item() * images.size(0)
            _, predicted = torch.max(predictions, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_loss = total_loss_test / total_test
    test_acc = 100 * correct_test / total_test

    # Sauvegarder l'historique
    historique['train_loss'].append(train_loss)
    historique['train_acc'].append(train_acc)
    historique['test_loss'].append(test_loss)
    historique['test_acc'].append(test_acc)

    print(f"{epoch+1:>6} | {train_loss:>12.4f} | {train_acc:>9.2f}% | {test_loss:>12.4f} | {test_acc:>9.2f}%")

print("=" * 70)
print("Entrainement termine !")

# === Visualiser les courbes d'entrainement ===
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Courbe de perte
ax1.plot(range(1, NB_EPOCHS+1), historique['train_loss'], 'b-o', label='Train', linewidth=2)
ax1.plot(range(1, NB_EPOCHS+1), historique['test_loss'], 'r-o', label='Test', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Perte (Loss)')
ax1.set_title('Evolution de la perte')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Courbe de precision
ax2.plot(range(1, NB_EPOCHS+1), historique['train_acc'], 'b-o', label='Train', linewidth=2)
ax2.plot(range(1, NB_EPOCHS+1), historique['test_acc'], 'r-o', label='Test', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Precision (%)')
ax2.set_title('Evolution de la precision')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle('Courbes d\'entrainement du MLP sur MNIST', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## Section 5 - Evaluation

Evaluons notre modele en detail :
- **Precision globale** sur le jeu de test
- **Matrice de confusion** pour voir quels chiffres sont confondus
- **Visualisation** de predictions correctes et incorrectes

In [ ]:
# === Evaluation detaillee sur le jeu de test ===

modele.eval()
toutes_predictions = []
tous_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        sorties = modele(images)
        _, predictions = torch.max(sorties, 1)
        toutes_predictions.extend(predictions.cpu().numpy())
        tous_labels.extend(labels.numpy())

toutes_predictions = np.array(toutes_predictions)
tous_labels = np.array(tous_labels)

# Precision globale
precision_globale = 100 * (toutes_predictions == tous_labels).sum() / len(tous_labels)
print(f"Precision globale sur le jeu de test : {precision_globale:.2f}%")

# Precision par classe
print(f"\n{'Chiffre':>8} | {'Precision':>10} | {'Nb correct':>11} / {'Total':>6}")
print("-" * 50)
for chiffre in range(10):
    masque = tous_labels == chiffre
    nb_correct = (toutes_predictions[masque] == chiffre).sum()
    nb_total = masque.sum()
    precision = 100 * nb_correct / nb_total
    print(f"{chiffre:>8} | {precision:>9.1f}% | {nb_correct:>11} / {nb_total:>6}")

# === Matrice de confusion ===
cm = confusion_matrix(tous_labels, toutes_predictions)

fig, ax = plt.subplots(figsize=(10, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(10))
disp.plot(ax=ax, cmap='Blues', values_format='d')
ax.set_title('Matrice de confusion - MLP sur MNIST', fontsize=14, fontweight='bold')
ax.set_xlabel('Prediction')
ax.set_ylabel('Verite')
plt.tight_layout()
plt.show()

In [ ]:
# === Visualisation de predictions ===

# Recuperer un batch d'images de test
images_test, labels_test = next(iter(test_loader))
images_test_device = images_test.to(device)

modele.eval()
with torch.no_grad():
    sorties = modele(images_test_device)
    probas = torch.softmax(sorties, dim=1)  # Convertir les logits en probabilites
    _, preds = torch.max(sorties, 1)

preds = preds.cpu()
probas = probas.cpu()

# Separer les predictions correctes et incorrectes
corrects = (preds == labels_test).nonzero(as_tuple=True)[0]
incorrects = (preds != labels_test).nonzero(as_tuple=True)[0]

# --- Predictions CORRECTES ---
fig, axes = plt.subplots(2, 5, figsize=(15, 7))
fig.suptitle('Predictions CORRECTES', fontsize=16, fontweight='bold', color='green')

for i, ax in enumerate(axes.flat):
    if i < len(corrects):
        idx = corrects[i].item()
        img = images_test[idx].squeeze()
        ax.imshow(img.numpy(), cmap='gray')
        confiance = probas[idx][preds[idx]].item() * 100
        ax.set_title(f'Pred: {preds[idx].item()} ({confiance:.0f}%)', fontsize=11, color='green')
    ax.axis('off')
plt.tight_layout()
plt.show()

# --- Predictions INCORRECTES ---
if len(incorrects) > 0:
    nb_incorrects_affichees = min(10, len(incorrects))
    fig, axes = plt.subplots(2, 5, figsize=(15, 7))
    fig.suptitle('Predictions INCORRECTES', fontsize=16, fontweight='bold', color='red')

    for i, ax in enumerate(axes.flat):
        if i < len(incorrects):
            idx = incorrects[i].item()
            img = images_test[idx].squeeze()
            ax.imshow(img.numpy(), cmap='gray')
            confiance = probas[idx][preds[idx]].item() * 100
            ax.set_title(
                f'Pred: {preds[idx].item()} ({confiance:.0f}%)\nVrai: {labels_test[idx].item()}',
                fontsize=10, color='red'
            )
        ax.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("Aucune prediction incorrecte dans ce batch ! Le modele est performant.")

---
## Section 6 - Ameliorations

Ameliorons notre modele avec deux techniques courantes :

### Dropout
Le **Dropout** desactive aleatoirement des neurones pendant l'entrainement (typiquement 20-50% des neurones). Cela force le reseau a ne pas dependre d'un seul neurone et ameliore la **generalisation**.

### Batch Normalization
La **Batch Normalization** normalise les activations de chaque couche, ce qui :
- Stabilise et accelere l'entrainement
- Permet d'utiliser des taux d'apprentissage plus eleves
- A un leger effet de regularisation

In [ ]:
# === MLP Ameliore avec Dropout et BatchNorm ===

class MLPAmeliore(nn.Module):
    """MLP ameliore avec Dropout et Batch Normalization."""

    def __init__(self, dropout_rate=0.3):
        super(MLPAmeliore, self).__init__()

        self.couches = nn.Sequential(
            nn.Flatten(),

            nn.Linear(784, 128),
            nn.BatchNorm1d(128),     # Normalisation par batch
            nn.ReLU(),
            nn.Dropout(dropout_rate), # Dropout : desactive 30% des neurones

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(64, 10),
        )

    def forward(self, x):
        return self.couches(x)


# Instancier le modele ameliore
modele_ameliore = MLPAmeliore(dropout_rate=0.3).to(device)
print(modele_ameliore)

nb_params_ameliore = sum(p.numel() for p in modele_ameliore.parameters())
print(f"\nNombre de parametres : {nb_params_ameliore:,} (vs {nb_params:,} pour le MLP simple)")

# === Entrainement du modele ameliore ===
critere_ameliore = nn.CrossEntropyLoss()
optimiseur_ameliore = optim.Adam(modele_ameliore.parameters(), lr=0.001)

historique_ameliore = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': []}

print("\n" + "=" * 70)
print(f"{'Epoch':>6} | {'Train Loss':>12} | {'Train Acc':>10} | {'Test Loss':>12} | {'Test Acc':>10}")
print("=" * 70)

for epoch in range(NB_EPOCHS):
    # Entrainement
    modele_ameliore.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        predictions = modele_ameliore(images)
        loss = critere_ameliore(predictions, labels)
        optimiseur_ameliore.zero_grad()
        loss.backward()
        optimiseur_ameliore.step()

        total_loss += loss.item() * images.size(0)
        _, predicted = torch.max(predictions, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = total_loss / total
    train_acc = 100 * correct / total

    # Evaluation
    modele_ameliore.eval()
    total_loss_test, correct_test, total_test = 0, 0, 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            predictions = modele_ameliore(images)
            loss = critere_ameliore(predictions, labels)
            total_loss_test += loss.item() * images.size(0)
            _, predicted = torch.max(predictions, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_loss = total_loss_test / total_test
    test_acc = 100 * correct_test / total_test

    historique_ameliore['train_loss'].append(train_loss)
    historique_ameliore['train_acc'].append(train_acc)
    historique_ameliore['test_loss'].append(test_loss)
    historique_ameliore['test_acc'].append(test_acc)

    print(f"{epoch+1:>6} | {train_loss:>12.4f} | {train_acc:>9.2f}% | {test_loss:>12.4f} | {test_acc:>9.2f}%")

print("=" * 70)

# === Comparaison des deux modeles ===
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Perte
ax1.plot(range(1, NB_EPOCHS+1), historique['test_loss'], 'r-o', label='MLP Simple (test)', linewidth=2)
ax1.plot(range(1, NB_EPOCHS+1), historique_ameliore['test_loss'], 'g-s', label='MLP Ameliore (test)', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Perte (Loss)')
ax1.set_title('Comparaison de la perte sur le test')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Precision
ax2.plot(range(1, NB_EPOCHS+1), historique['test_acc'], 'r-o', label='MLP Simple (test)', linewidth=2)
ax2.plot(range(1, NB_EPOCHS+1), historique_ameliore['test_acc'], 'g-s', label='MLP Ameliore (test)', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Precision (%)')
ax2.set_title('Comparaison de la precision sur le test')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle('MLP Simple vs MLP Ameliore (Dropout + BatchNorm)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Resume
print(f"\n{'Modele':<20} | {'Precision test finale':>22}")
print("-" * 46)
print(f"{'MLP Simple':<20} | {historique['test_acc'][-1]:>21.2f}%")
print(f"{'MLP Ameliore':<20} | {historique_ameliore['test_acc'][-1]:>21.2f}%")

---
## Conclusion

### Points cles a retenir

| Concept | Description |
|---|---|
| **Tensor** | Structure de donnees fondamentale de PyTorch, similaire a un array NumPy mais compatible GPU |
| **nn.Module** | Classe de base pour definir un reseau de neurones en PyTorch |
| **Forward pass** | Propagation des donnees a travers le reseau |
| **Backward pass** | Calcul des gradients par retropropagation |
| **CrossEntropyLoss** | Fonction de perte standard pour la classification multi-classes |
| **Adam** | Optimiseur adaptatif, bon choix par defaut |
| **Dropout** | Regularisation par desactivation aleatoire de neurones |
| **BatchNorm** | Normalisation des activations pour stabiliser l'entrainement |

### Pour aller plus loin

- Essayez de modifier l'architecture (plus de couches, plus de neurones)
- Testez differents taux d'apprentissage (lr)
- Remplacez le MLP par un **CNN** (Convolutional Neural Network) pour de meilleures performances
- Explorez d'autres datasets : Fashion-MNIST, CIFAR-10

> **Prochain notebook** : Nous verrons en detail l'**overfitting** et les techniques de **regularisation**.